In [5]:
import os
import subprocess as sp
import itertools
import librosa
import pickle
from pyAudioAnalysis import audioFeatureExtraction
import numpy as np
from keras.preprocessing import sequence
from scipy import stats
import sys

In [75]:
class Dataset:
    def __init__(self, path, dataset, decode=False):
        self.dataset = dataset
        if dataset == "berlin":
#             self.classes = {0: 'W', 1: 'L', 2: 'E', 3: 'A', 4: 'F', 5: 'T', 6: 'N'}
            self.classes = {0: 'W', 1: 'A', 2: 'F', 3: 'T', 4: 'N'}
#             self.classes = {0: 'W', 1: 'A', 2: 'F', 3: 'T', 4: 'N'}
            self.get_berlin_dataset(path)
        elif dataset == "dafex":
#             self.classes = {0: 'ang', 2: 'dis', 3: 'fea', 4: 'hap', 5: 'sad', 6: 'neu',  7: 'sur'}
            self.classes = {0: 'ang', 1: 'fea', 2: 'hap', 3: 'sad', 4: 'neu',}
            self.get_dafex_dataset(path)
        elif dataset == 'CASIA':
#             self.classes = {0: 'angry', 2: 'fear', 3: 'happy', 4: 'neutral', 5: 'sad', 6: 'surprise'}
#             self.classes = {0: 'angry', 1: 'fear', 2: 'happy',  3: 'sad', 4: 'neu'}
#             self.classes = {0: 'angry', 3: 'fear', 4: 'happy', 5: 'sad', 6: 'neu',  7: 'surprise'}
            self.classes = {0: 'angry', 1: 'fear', 2: 'happy', 3: 'sad', 4: 'neu'}
            self.get_CASIA_dataset(path)
    
    def get_berlin_dataset(self, path):
        males = ['03', '10', '11', '12', '15']
        females = ['08', '09', '13', '14', '16']
        test=['15','16']
        classes = {v:k for k,v in self.classes.items()}
        self.targets = []
        self.data = []
        self.train_sets = []
        self.test_sets = []
        self.val = []
        get_data = True
        for speak_test in itertools.product(males, females):  # test_couples:
            i = 0
            train = []
            test = []
            for audio in os.listdir(path):
                    print(audio)
#                     print(audio[:2])
#                 if audio[5]!='L' and audio[5]!='E':
                    audio_path = os.path.join(path, audio)
                    y, sr = librosa.load(audio_path, sr=16000)
                    if get_data:
                        try:
                            self.targets.append(classes[audio[5]])
                            self.data.append((y, sr))
                        except:
                            pass
#                     if audio[:2] in speak_test:
                    if audio[:2] == '15'or audio[:2] == '16':
                        print('test')
                        try:
                            self.test_sets.append([(y,sr),classes[audio[5]]])
                        except:
                            pass
#                         test.append(i)
                    else:
#                         train.append(i)
                        try:
                            self.train_sets.append([(y,sr),classes[audio[5]]])
                        except:
                            pass
                    i = i + 1
#             self.train_sets.append(train)
#             self.test_sets.append(test)
            get_data = False
    
    def get_dafex_dataset(self, path):
        classes = {v:k for k,v in self.classes.items()}
        self.targets = []
        self.data = []
        self.train_sets = []
        self.test_sets = []
        for actor_dir in os.listdir(path):
#             print('*******************1')
            if actor_dir[-1].isdigit():  # avoid invisible files
                    print(actor_dir, actor_dir[-1])
                    actor_path = os.path.join(path, actor_dir)
                    for block in os.listdir(actor_path):
                        if block[-1].isdigit():
#                             print(2)
                            block_path = os.path.join(actor_path, block)
                        for f_video in os.listdir(block_path):
#                             print(3)
                            if f_video.endswith('wav'):  # avoid invisible files
                                ss = f_video.split()
                                audio_path = os.path.join(block_path, f_video)
                                y, sr = librosa.load(audio_path.replace("\ ", " "), sr=16000)
                                try:
                                    if actor_dir[-1]=='8' or actor_dir[-1]=='6':
                                        self.test_sets.append([(y,sr),classes[ss[6]]])
                                    else:    
                                        self.train_sets.append([(y,sr),classes[ss[6]]])
                                    self.targets.append(classes[ss[6]])  # getting targets
                                    self.data.append((y, sr))  # getting signals + sr
                                except:
                                    continue
                            
    def get_dafex_dataset_old(self, path):
        males = ['4', '5', '7', '8']
        females = ['1', '2', '3', '6']
        no_audio = [3, 6]
        classes = {v: k for k, v in self.classes.items()}
        self.targets = []
        self.data = []
        self.train_sets = []
        self.test_sets = []
        get_data = True
        for speak_test in itertools.product(males, females):
            i = 0
            train = []
            test = []
            for actor_dir in os.listdir(path):
#                 print('actor_dir',actor_dir)
                if actor_dir[-1].isdigit():  # avoid invisible files
                    actor_path = os.path.join(path, actor_dir)
                    print('actor_path',actor_path)
                    for block in os.listdir(actor_path):
                        
                        if block[-1].isdigit() and int(
                                block[-1]) not in no_audio:  # avoid only video blocks and invisible files
                            block_path = os.path.join(actor_path, block)
                            for f_video in os.listdir(block_path):
                                print('f_video',f_video)
                                if f_video.endswith('avi') or f_video.endswith('wav'):  # avoid invisible files
                                    ss = f_video.split()
                                    print('ss',ss)
#                                     f_video = f_video.replace(" ", "\ ")  # for shell command
                                    video_path = os.path.join(block_path, f_video)
                                    print('v',video_path)
                                    audio_path = video_path.replace('.avi', '.wav')  # output
#                                     print('a',audio_path)
#                                     os.rename(video_path,audio_path)
                                    if decode and get_data:
                                        sp.call("ffmpeg -i " + video_path +
                                                " -ab 160k -ac 1 -ar 22050 -vn " + audio_path, shell=True)
                                    y, sr = librosa.load(audio_path.replace("\ ", " "), sr=16000)
                                    
                                    if get_data:
                                        try:
                                            self.targets.append(classes[ss[6]])  # getting targets
                                            self.data.append((y, sr))  # getting signals + sr
                                        except:
                                            continue
                                        
                                    if actor_dir[-1] in speak_test:
                                        test.append(i)
                                    else:
                                        train.append(i)
                                    i = i + 1
#             self.train_sets.append(train)
#             self.test_sets.append(test)
            get_data = False
            
    def get_CASIA_dataset(self, path):
        classes = {v:k for k,v in self.classes.items()}
        self.targets = []
        self.data = []
        self.train_sets = []
        self.test_sets = []
        for file in os.listdir(path):
            for file_dir in os.listdir(path+file):
#                 if file_dir!='surprise': 
#                     print(file_dir)
                    for filename in os.listdir(path+file+'/'+file_dir):
                        if filename.endswith('.wav') or filename.endswith('.WAV'):
                            audio_path = path+file+'/'+file_dir+'/'+filename
                            y, sr = librosa.load(audio_path, sr=16000)
#                             print('--------------')
#                             la = classes[file_dir]
#                             print(la)
                            try:
        
                                self.targets.append(classes[file_dir])
                                self.data.append((y, sr))
                            except:
                                pass

In [113]:
def pickle_dataset(path, dataset):
    ds=Dataset(path=path, dataset=dataset)
    print("Writing " + dataset + " data set to file...")
    pickle.dump(ds, open(dataset + '_db.p', 'wb'))
    print('Done')
    print("Extracting features and set to file...")
    f_globle = []
    for (x, Fs) in ds.data:
        f = audioFeatureExtraction.stFeatureExtraction(x, Fs, 0.050 * Fs, 0.025 * Fs)
        if type(f) is tuple:
            f = f[0]
        f = stats.zscore(f, axis=0)
        f = f.transpose()
        f_globle.append(f)
    f_globle = sequence.pad_sequences(f_globle,
                                     maxlen=516, dtype='float32', padding='post', value=-100)
    
    for (x, Fs) in ds.train_sets:
        x=x[0]
        f = audioFeatureExtraction.stFeatureExtraction(x, Fs, 0.050 * Fs, 0.025 * Fs)
        if type(f) is tuple:
            f = f[0]
        f = stats.zscore(f, axis=0)
        f = f.transpose()
        f_globle.append(f)
    f_globle = sequence.pad_sequences(f_globle,
                                     maxlen=516, dtype='float32', padding='post', value=-100)
    pickle.dump(f_globle, open(dataset + 'tra_features_ActorDep.p', 'wb'))
    for (x, Fs) in ds.test_sets:
        x=x[0]
        f = audioFeatureExtraction.stFeatureExtraction(x[0], Fs, 0.050 * Fs, 0.025 * Fs)
        if type(f) is tuple:
            f = f[0]
        f = stats.zscore(f, axis=0)
        f = f.transpose()
        f_globle.append(f)
    f_globle = sequence.pad_sequences(f_globle,
                                     maxlen=516, dtype='float32', padding='post', value=-100)
    pickle.dump(f_globle, open(dataset + 'val_features_ActorDep.p', 'wb'))
    print('Done')

In [114]:
def load_data(dataset):
    if dataset=='berlin':
        data_p = Dataset(path='./Berlin/wav/', dataset='berlin')
        data_p = pickle.load(open(dataset+'_db.p', 'rb'))
        features_p = pickle.load(open(dataset+'_features_ActorDep.p', 'rb'))
    
    if dataset=='dafex':
        data_p = Dataset(path='./DaFEx/', dataset='dafex')
        data_p = pickle.load(open(dataset+'_db.p', 'rb'))
        features_p = pickle.load(open(dataset+'_features_ActorDep.p', 'rb'))
        
    if dataset=='CASIA':
        data_p = Dataset(path='./CASIA/', dataset='CASIA')
        data_p = pickle.load(open(dataset+'_db.p', 'rb'))
        features_p = pickle.load(open(dataset+'_features_ActorDep.p', 'rb'))
    
    
    return data_p, features_p

def attention_init(train_len, test_len, attention_para, init_value):
    u_train = np.full((train_len, attention_para),
                      init_value, dtype=np.float32)
    u_test = np.full((test_len, attention_para),
                     init_value, dtype=np.float32)
    return u_train, u_test

In [115]:
def data_concatenate(data1,data2,label1,label2,shuffle=False):
    mix = []
    mix_data = []
    mix_label = []
    for i in range(len(data1)):
        mix.append([data1[i], label1[i]])
    for i in range(len(data2)):
        mix.append([data2[i], label2[i]])
    if shuffle:
        np.random.shuffle(mix)
        for i in mix:
            mix_data.append(i[0])
            mix_label.append(i[1])
    else:
        for i in mix:
            mix_data.append(i[0])
            mix_label.append(i[1])
    return np.asarray(mix_data), np.asarray(mix_label)

In [116]:
def pkl_data(data,label,dataname):
    from sklearn.model_selection import train_test_split
    from keras.utils import to_categorical
    import pickle
    label = to_categorical(label, num_classes=5)
    tra_data, val_data, tra_label, val_label = train_test_split(data, label, test_size=0.3, random_state=886)
    #     tra_data_mix, tra_label_mix = data_concatenate(tra_data1,tra_data2,tra_label1,tra_label2,shuffle=True)
    #     val_data_mix, val_label_mix = data_concatenate(val_data1,val_data2,val_label1,val_label2,shuffle=True)
    pickle.dump(tra_data, open('tra_data_5cla_'+ dataname + '_features_ActorDep.p', 'wb'))
    pickle.dump(tra_label, open('tra_label_5cla_' + dataname + '_labels_ActorDep.p', 'wb'))
    pickle.dump(val_data, open('val_data_5cla_' + dataname + '_features_ActorDep.p', 'wb'))
    pickle.dump(val_label, open('val_label_5cla_' + dataname + '_labels_ActorDep.p', 'wb'))
    
def pkl_data_mix(data1,label1,data2,label2,dataname):
    from sklearn.model_selection import train_test_split
    from keras.utils import to_categorical
    import pickle
    label1 = to_categorical(label1, num_classes=5)
    label2 = to_categorical(label2, num_classes=5)
    tra_data1, val_data1, tra_label1, val_label1 = train_test_split(data1, label1, test_size=0.3, random_state=886)
    tra_data2, val_data2, tra_label2, val_label2 = train_test_split(data2, label2, test_size=0.3, random_state=886)
    tra_data_mix, tra_label_mix = data_concatenate(tra_data1,tra_data2,tra_label1,tra_label2,shuffle=True)
    val_data_mix, val_label_mix = data_concatenate(val_data1,val_data2,val_label1,val_label2,shuffle=True)
    pickle.dump(tra_data_mix, open('tra_data_5cla_'+ dataname + '_features.p', 'wb'))
    pickle.dump(tra_label_mix, open('tra_label_5cla_' + dataname + '_labels.p', 'wb'))
    pickle.dump(val_data_mix, open('val_data_5cla_' + dataname + '_features.p', 'wb'))
    pickle.dump(val_label_mix, open('val_label_5cla_' + dataname + '_labels.p', 'wb'))

In [117]:
if __name__=='__main__':
    pickle_dataset(path='./Berlin/wav', dataset='berlin')
    pickle_dataset(path='./DaFEx', dataset='dafex')
#     pickle_dataset(path='./CASIA/', dataset='CASIA')

14a05Tc.wav
15a01Ea.wav
test
14b10Lb.wav
13a05Nb.wav
13b03Wc.wav
09a05Nb.wav
14a01Na.wav
08b10Tc.wav
16a02Lb.wav
test
16a04Wc.wav
test
09a04Wa.wav
12b02Na.wav
11b10Ld.wav
15a07Eb.wav
test
15a02Na.wav
test
16a01Tb.wav
test
15a04Wa.wav
test
15a05Wa.wav
test
12b03Ta.wav
12a04Wc.wav
16b03Fa.wav
test
11a01Ld.wav
08b09Nb.wav
11a05Wd.wav
13b10La.wav
08b10Aa.wav
11a07Wc.wav
15b01Ec.wav
test
03a04Ta.wav
10a05Aa.wav
03a07La.wav
15a07Ld.wav
test
16b01La.wav
test
14a02Wc.wav
14b10Eb.wav
15b01Wc.wav
test
09b03Lb.wav
16b09Wb.wav
test
03a02Nc.wav
11b01Ab.wav
16b03La.wav
test
11b03Fc.wav
13b10Wa.wav
11b03Nb.wav
08b09Tb.wav
15a02Ta.wav
test
09b10Wa.wav
13a02Ta.wav
11a05Fc.wav
15b03Lc.wav
test
13a07Na.wav
13b10Fa.wav
15b09Wb.wav
test
14a02Nc.wav
14a01Ac.wav
14b09Fc.wav
03b09Wa.wav
14a04Ed.wav
08b10Nc.wav
16b01Fa.wav
test
09a02Ea.wav
10b03Tb.wav
15b09Ac.wav
test
11b01Fc.wav
10b10Wa.wav
03b10Wc.wav
09b02Wc.wav
03b01Wc.wav
16b09Fb.wav
test
14a02Wa.wav
16b03Fd.wav
test
11a01Nd.wav
12b02Ea.wav
16a05Ea.wav
te

08b02La.wav
14a01Ea.wav
03b10Ec.wav
11a04Nd.wav
15a04Ac.wav
test
11b10Ae.wav
08a01Wa.wav
08b02Nb.wav
12b03La.wav
13b03Ed.wav
15a07Fa.wav
test
08a07Tb.wav
09b10Aa.wav
11b09Na.wav
03b03Wc.wav
15a05Fb.wav
test
14a02La.wav
13b03Ac.wav
03b09Nc.wav
16a05Tb.wav
test
08b01Lb.wav
11b02Wb.wav
08a02Ac.wav
16b09Ab.wav
test
15b03Tc.wav
test
12a02Ec.wav
11a02Fb.wav
16a05Wb.wav
test
11a07Ld.wav
08b10La.wav
12b01Wa.wav
10a05Ld.wav
13a02Ec.wav
10b03Wb.wav
11b02Ab.wav
10b01Lb.wav
11b02Na.wav
14a05Wb.wav
10a04Nb.wav
16a02Wb.wav
test
14b09Ea.wav
13b02Nb.wav
10a02Fa.wav
03b02Na.wav
09b09Nd.wav
09a04La.wav
10b02Aa.wav
09b03Wb.wav
08b03Wd.wav
08a05Lc.wav
10a02Wa.wav
12a07Wa.wav
16a07Wa.wav
test
16a07Nb.wav
test
10b01Fa.wav
16a07Ea.wav
test
16a04Nc.wav
test
13a02Lc.wav
16b10Aa.wav
test
11b01Wd.wav
16b01Wb.wav
test
09a05Tb.wav
12a02Wc.wav
13a05Tc.wav
16a01Ec.wav
test
13b01Ab.wav
11b03Wb.wav
16b03Wb.wav
test
08a05Nb.wav
11a02Wc.wav
16b03Nb.wav
test
15a05Eb.wav
test
16a04Tc.wav
test
16b02Lb.wav
test
14a02Fd.wav


09b03Ta.wav
10b09Wb.wav
08b01Fe.wav
16b01Wa.wav
test
08b10Fd.wav
11b10Ad.wav
13a02Wa.wav
14a07Tc.wav
13b10Wc.wav
14b09Td.wav
03a07Nc.wav
15a07Ac.wav
test
12b09Ac.wav
11a07Ta.wav
16b10Td.wav
test
14a05Na.wav
14a02Ab.wav
12b02Wb.wav
15a07Nc.wav
test
13a04Wc.wav
13b01Fc.wav
10a02Na.wav
03b10Ab.wav
08a07Na.wav
03a01Fa.wav
09a01Nb.wav
14a07Eb.wav
12b02Wd.wav
10a04Wa.wav
10a01Ac.wav
12a05Lb.wav
11b09Wa.wav
13a07Fd.wav
16b02Fd.wav
test
09b03Fd.wav
15b01Lb.wav
test
14a04Wc.wav
15b02Aa.wav
test
15a04Nc.wav
test
16a01Fc.wav
test
15b02Nd.wav
test
14b03Wb.wav
14a05Lb.wav
16a07La.wav
test
13a01Ec.wav
11b09Ad.wav
16a04Wb.wav
test
16a01Lb.wav
test
08a02Ab.wav
13a02Ad.wav
10b02La.wav
10b01Ea.wav
15b02Wc.wav
test
12a05Nd.wav
15b03Wa.wav
test
16b03Ea.wav
test
13b01Ld.wav
12b01Ta.wav
15b02Lb.wav
test
03b09La.wav
08b09Fd.wav
13b03Lb.wav
08a04Wc.wav
16a01Nc.wav
test
13a05Wc.wav
03a05Tc.wav
03b02Tb.wav
09b02Wd.wav
03b03Tc.wav
03b10Na.wav
10a05Wb.wav
16b10Lb.wav
test
14b09Lb.wav
11a05Lc.wav
15a02La.wav
test


03a04Wc.wav
08a01Fd.wav
10b10Fc.wav
15b10Ac.wav
test
15a02Wd.wav
test
16b09Eb.wav
test
16a05Ab.wav
test
08b01Aa.wav
12a01Lb.wav
11a02Ec.wav
16b02Wb.wav
test
03a05Wa.wav
11b03Wa.wav
16a07Lb.wav
test
13b02Wa.wav
08a01Ab.wav
14a05Ac.wav
14b01Fa.wav
16b01Aa.wav
test
13b09Wa.wav
08a04Ff.wav
14b09Wc.wav
12b10Wa.wav
14a07Lc.wav
11a04Wc.wav
11b10Nc.wav
16b01Lc.wav
test
03a02Wc.wav
11b10Td.wav
15b10Wa.wav
test
14b01Fc.wav
12b02Wa.wav
16a07Fa.wav
test
11a02Tc.wav
11a01Aa.wav
14b01Na.wav
12a01Nb.wav
08b09Ab.wav
09b01Na.wav
08a02La.wav
09a01Ea.wav
13b09Ab.wav
03b09Tc.wav
14b02Wb.wav
15b02Wa.wav
test
15b09Ta.wav
test
13a05Lc.wav
11a02Nc.wav
14b10Nb.wav
14a02Ea.wav
13b01Ec.wav
16a04Ab.wav
test
15a04Wb.wav
test
09a05Lc.wav
09a07Wb.wav
08b03Fe.wav
09a02La.wav
03a01Wa.wav
13a01Ac.wav
08a07Wc.wav
03b01Wa.wav
12a07Ac.wav
13a01Ea.wav
09a07Na.wav
15a01Fb.wav
test
14a07Aa.wav
16b10Fb.wav
test
08a04Nc.wav
11b01Nc.wav
08b10Wa.wav
16a07Fb.wav
test
08a07Fd.wav
12a02Ac.wav
14b02Aa.wav
09b01Wb.wav
15b09Fa.wav
tes

11a05Wd.wav
13b10La.wav
08b10Aa.wav
11a07Wc.wav
15b01Ec.wav
test
03a04Ta.wav
10a05Aa.wav
03a07La.wav
15a07Ld.wav
test
16b01La.wav
test
14a02Wc.wav
14b10Eb.wav
15b01Wc.wav
test
09b03Lb.wav
16b09Wb.wav
test
03a02Nc.wav
11b01Ab.wav
16b03La.wav
test
11b03Fc.wav
13b10Wa.wav
11b03Nb.wav
08b09Tb.wav
15a02Ta.wav
test
09b10Wa.wav
13a02Ta.wav
11a05Fc.wav
15b03Lc.wav
test
13a07Na.wav
13b10Fa.wav
15b09Wb.wav
test
14a02Nc.wav
14a01Ac.wav
14b09Fc.wav
03b09Wa.wav
14a04Ed.wav
08b10Nc.wav
16b01Fa.wav
test
09a02Ea.wav
10b03Tb.wav
15b09Ac.wav
test
11b01Fc.wav
10b10Wa.wav
03b10Wc.wav
09b02Wc.wav
03b01Wc.wav
16b09Fb.wav
test
14a02Wa.wav
16b03Fd.wav
test
11a01Nd.wav
12b02Ea.wav
16a05Ea.wav
test
12a05Ta.wav
03a05Wb.wav
09a04Nb.wav
09a02Eb.wav
16b03Ad.wav
test
14b01Eb.wav
09a04Fd.wav
14a07Fd.wav
14b09Ac.wav
09a05Ed.wav
10b09Ad.wav
10a02Ab.wav
08b02Wd.wav
11b03Lc.wav
09a07Wd.wav
08a05Fe.wav
03a07Wc.wav
16b10Tb.wav
test
09a07Ta.wav
13b02Fb.wav
15b03Nb.wav
test
08a04Tb.wav
15b10Nb.wav
test
12a05Wb.wav
14b01Ac.wa

...

12b01Wa.wav
10a05Ld.wav
13a02Ec.wav
10b03Wb.wav
11b02Ab.wav
10b01Lb.wav
11b02Na.wav
14a05Wb.wav
10a04Nb.wav
16a02Wb.wav
test
14b09Ea.wav
13b02Nb.wav
10a02Fa.wav
03b02Na.wav
09b09Nd.wav
09a04La.wav
10b02Aa.wav
09b03Wb.wav
08b03Wd.wav
08a05Lc.wav
10a02Wa.wav
12a07Wa.wav
16a07Wa.wav
test
16a07Nb.wav
test
10b01Fa.wav
16a07Ea.wav
test
16a04Nc.wav
test
13a02Lc.wav
16b10Aa.wav
test
11b01Wd.wav
16b01Wb.wav
test
09a05Tb.wav
12a02Wc.wav
13a05Tc.wav
16a01Ec.wav
test
13b01Ab.wav
11b03Wb.wav
16b03Wb.wav
test
08a05Nb.wav
11a02Wc.wav
16b03Nb.wav
test
15a05Eb.wav
test
16a04Tc.wav
test
16b02Lb.wav
test
14a02Fd.wav
11a05Ad.wav
09b02Tb.wav
13a07Tc.wav
08b03Nb.wav
03b01Nb.wav
13a01Wb.wav
14b03Ed.wav
13b09Na.wav
03a02Ta.wav
16a04Lc.wav
test
15a05Na.wav
test
13a04Ta.wav
15b01Na.wav
test
15a01Nb.wav
test
11a01Ab.wav
08b09Lc.wav
03b01Fa.wav
14b01Wc.wav
14b03Ta.wav
08a01Lc.wav
14a04Lb.wav
15b10Nc.wav
test
03a07Fb.wav
15b03Wb.wav
test
11a07Ac.wav
12b09Wc.wav
03a04Fd.wav
03b02La.wav
03b02Aa.wav
14a05Ta.wav
08a02

test
15b02Nd.wav
test
14b03Wb.wav
14a05Lb.wav
16a07La.wav
test
13a01Ec.wav
11b09Ad.wav
16a04Wb.wav
test
16a01Lb.wav
test
08a02Ab.wav
13a02Ad.wav
10b02La.wav
10b01Ea.wav
15b02Wc.wav
test
12a05Nd.wav
15b03Wa.wav
test
16b03Ea.wav
test
13b01Ld.wav
12b01Ta.wav
15b02Lb.wav
test
03b09La.wav
08b09Fd.wav
13b03Lb.wav
08a04Wc.wav
16a01Nc.wav
test
13a05Wc.wav
03a05Tc.wav
03b02Tb.wav
09b02Wd.wav
03b03Tc.wav
03b10Na.wav
10a05Wb.wav
16b10Lb.wav
test
14b09Lb.wav
11a05Lc.wav
15a02La.wav
test
14a04Tc.wav
08b02Ff.wav
15a07Fb.wav
test
13b01Nc.wav
13a01Lb.wav
08b02Tc.wav
08a02Fe.wav
15a02Wb.wav
test
09a01Wb.wav
13a02Nc.wav
09a07Eb.wav
16a04Ea.wav
test
12a01Fb.wav
14a01Wc.wav
15a02Ac.wav
test
12a02Wa.wav
12a01Wc.wav
12a05Ab.wav
13b09Fc.wav
08a01Na.wav
16b10Eb.wav
test
11a04Fd.wav
08a02Wc.wav
14a01Wa.wav
12a07La.wav
11a05Td.wav
13a05Ea.wav
16a02Nb.wav
test
03b03Nb.wav
16a02Ea.wav
test
11b09Fd.wav
14a05Aa.wav
03a04Ad.wav
11a01Wc.wav
16a05Wc.wav
test
09b02Na.wav
08a07Ta.wav
03b10Wb.wav
11b02Fd.wav
14a04Aa.wav


test
14a07Aa.wav
16b10Fb.wav
test
08a04Nc.wav
11b01Nc.wav
08b10Wa.wav
16a07Fb.wav
test
08a07Fd.wav
12a02Ac.wav
14b02Aa.wav
09b01Wb.wav
15b09Fa.wav
test
14b03Ad.wav
13b09Fb.wav
11a05Na.wav
08a07La.wav
09b03Nb.wav
16a05La.wav
test
08a05Ta.wav
14b02Fb.wav
13b09La.wav
16b09Lb.wav
test
03a05Aa.wav
14a01Aa.wav
09b01Ea.wav
15b09Nb.wav
test
08b09Wc.wav
09a01Fa.wav
13a04Fc.wav
14b02Na.wav
14a05Fa.wav
14a05Wa.wav
15b10Lc.wav
test
11b09Td.wav
11a02Ld.wav
13b03Fd.wav
13b01Wa.wav
16b10Wb.wav
test
03a07Fa.wav
16b03Ta.wav
test
03a01Nc.wav
16b01Tb.wav
test
14a07Ld.wav
14b02Wd.wav
12b02Fb.wav
08b01Wa.wav
03b01Lb.wav
08a02Na.wav
15a05Lb.wav
test
09b09Wa.wav
03b01Td.wav
10a05Tb.wav
14a05Tc.wav
15a01Ea.wav
test
14b10Lb.wav
13a05Nb.wav
13b03Wc.wav
09a05Nb.wav
14a01Na.wav
08b10Tc.wav
16a02Lb.wav
test
16a04Wc.wav
test
09a04Wa.wav
12b02Na.wav
11b10Ld.wav
15a07Eb.wav
test
15a02Na.wav
test
16a01Tb.wav
test
15a04Wa.wav
test
15a05Wa.wav
test
12b03Ta.wav
12a04Wc.wav
16b03Fa.wav
test
11a01Ld.wav
08b09Nb.wav
11a05Wd

test
12a05Wb.wav
14b01Ac.wav
03a05Fc.wav
08a01Wc.wav
15a04Fd.wav
test
03a04Lc.wav
16a04Fa.wav
test
15a01La.wav
test
11b01Lb.wav
11b01Eb.wav
13a04Ac.wav
13a01Fd.wav
09b09Ea.wav
14a07Wc.wav
13a05Aa.wav
09b10Nd.wav
10a04Wb.wav
12a02Nb.wav
16a02Tc.wav
test
03a02Wb.wav
14a04Wb.wav
11b10Wa.wav
03b02Wb.wav
13a04Lb.wav
08b03Lc.wav
10b02Na.wav
16a07Td.wav
test
10b03La.wav
10a07Ad.wav
15a01Wa.wav
test
10a07Ta.wav
13a05Wa.wav
10b10Lc.wav
10b01Aa.wav
14a02Tb.wav
08a04La.wav
13b10Nc.wav
11a04Ac.wav
03a02Fc.wav
08b03Tc.wav
08b02La.wav
14a01Ea.wav
03b10Ec.wav
11a04Nd.wav
15a04Ac.wav
test
11b10Ae.wav
08a01Wa.wav
08b02Nb.wav
12b03La.wav
13b03Ed.wav
15a07Fa.wav
test
08a07Tb.wav
09b10Aa.wav
11b09Na.wav
03b03Wc.wav
15a05Fb.wav
test
14a02La.wav
13b03Ac.wav
03b09Nc.wav
16a05Tb.wav
test
08b01Lb.wav
11b02Wb.wav
08a02Ac.wav
16b09Ab.wav
test
15b03Tc.wav
test
12a02Ec.wav
11a02Fb.wav
16a05Wb.wav
test
11a07Ld.wav
08b10La.wav
12b01Wa.wav
10a05Ld.wav
13a02Ec.wav
10b03Wb.wav
11b02Ab.wav
10b01Lb.wav
11b02Na.wav
14a05W

ValueError: Invalid number of FFT data points (0) specified.

In [ ]:
DATA_ger, features_ger = load_data('berlin')

In [ ]:
DATA_ita, features_ita = load_data('dafex')

# DATA_ch, features_ch = load_data('CASIA')

In [ ]:
import numpy as np
tra_data_ita=np.random.shuffle(DATA_ita.train_sets)
tra_data_ger=np.random.shuffle(DATA_ger.train_sets)
val_data_ita=np.random.shuffle(DATA_ita.test_sets)
val_data_ger=np.random.shuffle(DATA_ger.test_sets)

In [ ]:
tra_data=[]
tra_label=[]
val_data=[]
val_label=[]
for i in range(len(DATA_ita.train_sets)):
    tra_data.append(DATA_ita.train_sets[i][0])
    tra_label.append(DATA_ita.train_sets[i][1])
for i in range(len(DATA_ita.test_sets)):
    val_data.append(DATA_ita.test_sets[i][0])
    val_label.append(DATA_ita.test_sets[i][1])
for i in range(len(DATA_ger.train_sets)):
    tra_data.append(DATA_ger.train_sets[i][0])
    tra_label.append(DATA_ger.train_sets[i][1])
for i in range(len(DATA_ger.test_sets)):
    val_data.append(DATA_ger.test_sets[i][0])
    val_label.append(DATA_ger.test_sets[i][1])

In [96]:
print(tra_data)

[(array([-2.1289892e-05, -1.4010740e-06,  2.3305158e-06, ...,
       -3.2377432e-06,  1.1344974e-06, -3.5677920e-07], dtype=float32), 16000), (array([ 2.2111117e-05,  3.8449856e-05,  3.2233750e-06, ...,
       -5.4563716e-06, -2.9611267e-05, -1.5393754e-05], dtype=float32), 16000), (array([-1.3905451e-05,  1.8560990e-05, -2.4276007e-06, ...,
        2.6607271e-05, -3.3781089e-05, -1.2083523e-05], dtype=float32), 16000), (array([ 8.5284200e-06,  1.7748211e-05, -1.0553366e-06, ...,
        9.1586617e-06,  9.4266588e-06, -3.4649514e-05], dtype=float32), 16000), (array([-9.0095718e-06,  4.9264781e-05,  2.7941600e-05, ...,
        1.4691249e-05,  2.6788399e-05,  9.4233337e-06], dtype=float32), 16000), (array([-4.7907852e-05, -2.8240408e-06, -1.0096811e-05, ...,
       -7.8919460e-05, -6.0320664e-05, -5.1201401e-05], dtype=float32), 16000), (array([-2.1717033e-05,  1.3447875e-06, -1.9799474e-05, ...,
        9.8715873e-06,  1.7440529e-05, -3.1877079e-07], dtype=float32), 16000), (array([ 7.0

In [106]:
from keras.utils import to_categorical
tra_label = to_categorical(tra_label, num_classes=5)
val_label = to_categorical(val_label, num_classes=5)

In [107]:
dataname='mix'
pickle.dump(tra_data, open('tra_data_5cla_'+ dataname + '_features_ActorDep.p', 'wb'))
pickle.dump(tra_label, open('tra_label_5cla_' + dataname + '_labels_ActorDep.p', 'wb'))
pickle.dump(val_data, open('val_data_5cla_' + dataname + '_features_ActorDep.p', 'wb'))
pickle.dump(val_label, open('val_label_5cla_' + dataname + '_labels_ActorDep.p', 'wb'))

In [80]:
DATA_ger.test_sets[1][1]

2

In [14]:
pkl_data(features_ch,DATA_ch.targets,'ch')
pkl_data(features_ita,DATA_ita.targets,'ita')
pkl_data(features_ger,DATA_ger.targets,'ger')

In [17]:
pkl_data_mix(features_ita,DATA_ita.targets,features_ger,DATA_ger.targets,'mix')

In [99]:
ger_val_label=pickle.load(open('val_data_5cla_ger_features_ActorDep.p','rb'))

In [100]:
print(ger_val_label)

[(array([ 3.4456694e-05,  2.2876945e-06,  2.7383180e-06, ...,
        2.2681503e-05, -7.7528302e-06, -1.8330154e-05], dtype=float32), 16000), (array([-4.6510817e-05,  3.8644912e-06, -7.3592437e-06, ...,
        2.3600263e-05,  5.1842051e-07, -1.2651993e-05], dtype=float32), 16000), (array([ 7.6024944e-06, -1.9444882e-05, -6.4766796e-06, ...,
        7.7508766e-06, -1.4040558e-05, -1.4781846e-05], dtype=float32), 16000), (array([ 9.7717302e-06,  6.8914455e-06, -2.0482003e-05, ...,
        8.7041390e-06,  3.0795090e-05,  1.4058884e-05], dtype=float32), 16000), (array([ 2.0255497e-05,  3.2137450e-06, -2.0435604e-05, ...,
       -5.1633979e-06, -9.3137278e-06, -1.7228258e-05], dtype=float32), 16000), (array([-1.9816020e-05, -1.7249095e-06, -1.5212771e-05, ...,
       -3.8141723e-06, -3.0468897e-05, -1.5039205e-05], dtype=float32), 16000), (array([-3.2687585e-06,  1.3046277e-05,  1.6503689e-05, ...,
       -2.5467443e-06,  1.7558898e-05,  3.2851112e-05], dtype=float32), 16000), (array([-1.3